## [5. Longest Palindromic Substring](https://leetcode.com/problems/longest-palindromic-substring/)


### Brute Force

檢查所有子字串是不是回文

檢查回文 O(N)

Time: O(N^3)

### DP: Expand around Center

遍歷矩陣, 每次當作從中間元素開始, 向兩邊擴展

如果 s\[i:j] 是回文, s\[i+1,j-1] 才有可能是回文

雖然沒有直接用矩陣儲存結果

實際上等同於使用 dp\[i]\[j]: 檢查 i~j 是不是回文字串

dp\[i]]\[j] = dp\[i+1]\[j-1] and s\[i] == s\[j]

Time: best O(N), worst O(N^2)

.

In [ ]:
class Solution:
    def expand(self, s: str, l: int, r: int):
        while l >= 0 and r < len(s) and s[l] == s[r]:
            l -= 1
            r += 1
        return r - l - 1
    
    def longestPalindrome(self, s: str) -> str:
        if len(s) < 2 or s == s[::-1]:
            return s
        
        start = maxLen = 0

        for i in range(len(s)):
            odd = self.expand(s, i, i)
            even = self.expand(s, i, i+1)
            curLen = max(odd, even)
            if curLen > maxLen:
                maxLen = curLen
                start = i - (curLen-1) // 2
        return s[start:start + maxLen]

### [Manacher Algorithm](https://leetcode.com/problems/longest-palindromic-substring/discuss/3337/Manacher-algorithm-in-Python-O(n))

在字串中插入 # 解決奇數偶數的問題

使用 ^ 和 $ 標示頭尾, 避免擴展過頭

矩陣 P 儲存擴展的次數, 也會是最大長度

C: center, R: right edge

.

In [ ]:
class Solution:
    #Manacher algorithm
    #http://en.wikipedia.org/wiki/Longest_palindromic_substring
    
    def longestPalindrome(self, s):
        # Transform S into T.
        # For example, S = "abba", T = "^#a#b#b#a#$".
        # ^ and $ signs are sentinels appended to each end to avoid bounds checking
        T = '#'.join(f'^{s}$')
        n = len(T)
        P = [0] * n
        C = R = 0
        for i in range (1, n-1):
            P[i] = (R > i) and min(R - i, P[2*C - i]) # equals to i' = C - (i-C)
            # Attempt to expand palindrome centered at i
            while T[i + 1 + P[i]] == T[i - 1 - P[i]]:
                P[i] += 1
    
            # If palindrome centered at i expand past R,
            # adjust center based on expanded palindrome.
            if i + P[i] > R:
                C, R = i, i + P[i]
    
        # Find the maximum element in P.
        maxLen, centerIndex = max((n, i) for i, n in enumerate(P))
        return s[(centerIndex  - maxLen)//2: (centerIndex  + maxLen)//2]

In [1]:
s = 'abba'
T = '#'.join(f'^{s}$')
T

'^#a#b#b#a#$'

## [647. Palindromic Substrings](https://leetcode.com/problems/palindromic-substrings/)

計算回文子字串的總數

### Brute Force

檢查所有子字串, 看是不是回文

Time: O(N^3)

### DP: Expand around Center

Time: O(N^2)

In [ ]:
class Solution:
    def expand(self, s: str, l: int, r: int):
        ans = 0
        while l >= 0 and r < len(s) and s[l] == s[r]:
            l -= 1
            r += 1
            ans += 1
        return ans
    
    def countSubstrings(self, s: str) -> str:
        res = 0

        for i in range(len(s)):
            res += self.expand(s, i, i)
            res += self.expand(s, i, i+1)
        return res

## [336. Palindrome Pairs](https://leetcode.com/problems/palindrome-pairs/)

找到可以合成回文的所有組合, 順序

### Brute Force

測試所有組合是不是回文

Time: O(N^2 * k), k 最長字串的長度

### Hash Table

回文的組合有三種狀況

1. mid 在 word1 及 word2 正中間
    + word1 + word2

2. mid 在 word2 之中, word2 較長且包含 word1 的回文
    + word1 + palindrome + valid_suffix

3. mid 在 word1 之中, 構造為 
    + valid_prefix + palindrome + word2
.

In [ ]:
class Solution:
    def palindromePairs(self, words):

        def all_valid_prefixes(word):
            valid_prefixes = []
            for i in range(len(word)):
                if word[i:] == word[i:][::-1]:
                    valid_prefixes.append(word[:i])
            return valid_prefixes

        def all_valid_suffixes(word):
            valid_suffixes = []
            for i in range(len(word)):
                if word[:i+1] == word[:i+1][::-1]:
                    valid_suffixes.append(word[i + 1:])
            return valid_suffixes

        word_lookup = {word: i for i, word in enumerate(words)}
        solutions = []

        for word_index, word in enumerate(words):
            reversed_word = word[::-1]

            # Build solutions of case #1. This word will be word 1.
            if reversed_word in word_lookup and word_index != word_lookup[reversed_word]:
                solutions.append([word_index, word_lookup[reversed_word]])

            # Build solutions of case #2. This word will be word 2.
            for suffix in all_valid_suffixes(word):
                reversed_suffix = suffix[::-1]
                if reversed_suffix in word_lookup:
                    solutions.append([word_lookup[reversed_suffix], word_index])

            # Build solutions of case #3. This word will be word 1.
            for prefix in all_valid_prefixes(word):
                reversed_prefix = prefix[::-1]
                if reversed_prefix in word_lookup:
                    solutions.append([word_index, word_lookup[reversed_prefix]])

        return solutions


## [125. Valid Palindrome](https://leetcode.com/problems/valid-palindrome/)

### 2 pointers

傳換成小寫, 忽略非字母

In [ ]:
class Solution:
    def isPalindrome(self, s: str) -> bool:
        lo, hi = 0, len(s) - 1
        while lo < hi:
            while lo < hi and not s[lo].isalnum():
                lo += 1
            while lo < hi and not s[hi].isalnum():
                hi -= 1
            
            if lo < hi and s[lo].lower() != s[hi].lower():
                return False
            
            lo += 1
            hi -= 1
        
        return True

## [680. Valid Palindrome II](https://leetcode.com/problems/valid-palindrome-ii/)

最多可以刪除一個字母變成回文

### 2 Pointers

第一次遇到不符合的狀況, 可以跳過, 然後直接檢查剩下的字串是不是回文

.

In [ ]:
class Solution:
    def validPalindrome(self, s: str) -> bool:
        lo, hi = 0, len(s) - 1
        
        while lo < hi:
            if s[lo] != s[hi]:
                tmp1 = s[:lo] + s[lo+1:]
                tmp2 = s[:hi] + s[hi+1:]
                return tmp1 == tmp1[::-1] or tmp2 == tmp2[::-1]
                
            else:
                lo += 1
                hi -= 1
        
        return True